In [2]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pathlib import Path
from collections import defaultdict

# --- 1. Load Data with Polars ---
data_path = Path("data")
# NOTE: Assuming your data is in CSV format as per the scan_csv call.
# If they are Parquet files, you should use pl.scan_parquet.
train_data_in = data_path / "train" / "input*"
train_data_out = data_path / "train" / "output*"

game_id = 2023090700
play_id = 101

before_lf = pl.scan_csv(train_data_in).filter((pl.col("game_id") == game_id) & (pl.col("play_id") == play_id))
after_lf = pl.scan_csv(train_data_out).filter((pl.col("game_id") == game_id) & (pl.col("play_id") == play_id))

before = before_lf.collect()
after = after_lf.collect()

# --- 2. Convert Polars to Pandas ---
before_df = before.to_pandas()
after_df = after.to_pandas()


ball_landing_x = before_df['ball_land_x'].iloc[0]
ball_landing_y = before_df['ball_land_y'].iloc[0]

# --- 3. Animate the Data ---
players_to_predict = before_df[before_df['player_to_predict'] == True]['nfl_id'].unique()

# Dictionary to store player paths
paths = defaultdict(lambda: {'x': [], 'y': []})

# REMOVED: The pre-initialization of line objects and the 'lines' dictionary
# as they are cleared on each frame update.

# Lookup sides from before-data
side_lookup = before_df.set_index("nfl_id")["player_side"].to_dict()

# Last known positions of static players
last_known_positions = before_df.loc[before_df.groupby('nfl_id')['frame_id'].idxmax()]

# Timeline
before_frames = sorted(before_df['frame_id'].unique())

pass_release_frame = before_frames[-1]

after_frames = sorted(after_df['frame_id'].unique())
# We adjust the frame IDs of the 'after' data to ensure a continuous timeline
total_frames = before_frames + [f + pass_release_frame for f in after_frames]

fig, ax = plt.subplots(figsize=(12, 7))

def update(frame_id):
    ax.clear() # This clears the entire plot for a fresh draw

    ax.scatter(ball_landing_x, ball_landing_y, c='green', marker='x', s=150,
               label='Ball Landing Spot', zorder=10, linewidth=2.5)
    
    is_before_pass = frame_id <= pass_release_frame

    if is_before_pass:
        current_data = before_df[before_df['frame_id'] == frame_id]
        ax.set_title(f'Before Pass - Frame: {frame_id}')

        offense = current_data[current_data['player_side'] == 'Offense']
        defense = current_data[current_data['player_side'] == 'Defense']
        passer = offense[offense['player_role'] == 'Passer']
        receiver = offense[offense['player_role'] == 'Targeted Receiver']

        ax.scatter(defense['x'], defense['y'], c='red', marker='X', s=120, label='Defense')
        ax.scatter(offense['x'], offense['y'], c='gray', s=120, label='Offense')
        if not passer.empty:
            ax.scatter(passer['x'], passer['y'], c='orange', s=180, label='Passer',
                       edgecolors='black', zorder=5)
        if not receiver.empty:
            ax.scatter(receiver['x'], receiver['y'], c='blue', s=180, label='Receiver',
                       edgecolors='black', zorder=5)

    else:  # After the pass
        adjusted_frame = frame_id - pass_release_frame
        current_data = after_df[after_df['frame_id'] == adjusted_frame]
        ax.set_title(f'After Pass - Frame: {frame_id} (Tracing Paths)')

        # Static players
        static_players = last_known_positions[~last_known_positions['nfl_id'].isin(players_to_predict)]
        offense = static_players[static_players['player_side'] == 'Offense']
        defense = static_players[static_players['player_side'] == 'Defense']
        ax.scatter(defense['x'], defense['y'], c='red', marker='X', s=120,
                   label='Defense (Static)', alpha=0.3)
        ax.scatter(offense['x'], offense['y'], c='gray', s=120,
                   label='Offense (Static)', alpha=0.3)

        # Predicted players
        for pid in players_to_predict:
            player_pos = current_data[current_data['nfl_id'] == pid]
            if not player_pos.empty:
                x, y = player_pos['x'].iloc[0], player_pos['y'].iloc[0]
                paths[pid]['x'].append(x)
                paths[pid]['y'].append(y)

                # ADDED: Re-draw the entire path on each 'after' frame.
                # This works correctly with ax.clear().
                ax.plot(paths[pid]['x'], paths[pid]['y'], color='red', linewidth=2.5)

                side = side_lookup.get(pid, "Unknown")

                if side == "Defense":
                    ax.scatter(x, y, c='red', marker='X', s=120, zorder=5)
                else: # Default for Offense or Unknown
                    ax.scatter(x, y, c='blue', s=180, edgecolors='black', zorder=5)

    ax.set_xlim(0, 120)
    ax.set_ylim(0, 53.3)
    ax.set_xlabel('Yardline')
    ax.set_ylabel('Field Width (yards)')
    ax.grid(True)
    
    # De-duplicate legend
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')


# Create and save the animation
ani = animation.FuncAnimation(fig, update, frames=total_frames, repeat=False, interval=150)
ani.save(f'game_{game_id}_play_{play_id}.gif', writer='pillow')

plt.close(fig)
print(f"game_{game_id}_play_{play_id}.gif created successfully.")

game_2023090700_play_101.gif created successfully.
